<img height="60px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="20px" vspace="5px">

# Keras CNN Model

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/learning-stack/Colab-ML-Playbook/blob/master/Kaggle-State-Farm-Distracted-Driver-Detection/Keras%20CNN%20Model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/learning-stack/Colab-ML-Playbook/blob/master/Kaggle-State-Farm-Distracted-Driver-Detection/Keras%20CNN%20Model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [0]:
import os
import cv2
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from keras.utils import np_utils
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import random
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from sklearn.metrics import log_loss
from scipy.misc import imread, imresize
from keras.layers.convolutional import Convolution2D, MaxPooling2D
import keras
drivers=[]
images=[]
list_of_drivers=[]
list_of_image_names=[]
y_train=[]
x_train=[]
x_valid=[]
train_sample_size=5000
batch_size = 32
color_type=1
epochs = 20
data_augmentation = True
wigth=64
height=64

img_rows=img_cols=64

In [0]:
def return_drivers(go_path):
    train_path=os.path.join(go_path,'train/')
    drivers_file_names=os.listdir(train_path)
    for i in drivers_file_names:
        drivers.append(str(i))
    return drivers

In [0]:
def return_train_name_files(go_path,list_of_drivers):
    images=pd.read_csv(str(go_path)+'driver_imgs_list.csv')
    drivers_by_sort=images['classname']
    images_by_sort=images['img']
    return drivers_by_sort,images_by_sort

In [0]:
def main_call():
    y_valid=y_train=y_trainn=[]
    go_path="/home/beyhan/Kaggle/Araba_Pozisyon_Tahmin/imgs/"
    train_path="/home/beyhan/Kaggle/Araba_Pozisyon_Tahmin/imgs/train/"
    test_path="/home/beyhan/Kaggle/Araba_Pozisyon_Tahmin/imgs/testt/"
    list_of_drivers=return_drivers(str(go_path))
    drivers,images=return_train_name_files(str(go_path),list_of_drivers)
    y_valid,y_train=read_train_image(drivers,images,train_path,wigth,height)
    x_test=read_test_image(test_path,wigth,height)
    y_valid,y_trainn=read_train_image(drivers,images,train_path,wigth,height,valid=True)
    #show_image(image1)
    return y_valid,y_train

In [0]:
x_test=[]

In [0]:
def read_test_image(test_path,wigth,height):
    arr = os.listdir(str(test_path))
    for i in arr:
        fill_test(str(test_path)+str(i),wigth,height)

In [0]:
def fill_test(image_path,wigth,height):
    print(image_path)
    img=cv2.imread(image_path,0)
    rotate = random.uniform(-10, 10)
    M = cv2.getRotationMatrix2D((img.shape[1]/2, img.shape[0]/2), rotate, 1)
    img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    im = cv2.resize(img, (img_cols, img_rows), cv2.INTER_LINEAR)
    x_test.append(np.array(im, dtype=np.float32))


In [0]:
#read_test_image('/home/beyhan/Kaggle/Araba_Pozisyon_Tahmin/imgs/testt',64,64)

In [0]:
def read_train_image(all_drivers,images,train_path,wigth,height,valid=False):
    y_valid=y_trainn=y_train=[]
    total=len(all_drivers)
    if valid==False:
        for index,val in enumerate(all_drivers):
            if index<total*8/10:
            #if index<50:
             im_path=str(train_path)+str(val)+'/'+str(images[index])
             fill_train(im_path,wigth,height)
        y_valid,y_train=fill_train_target(all_drivers)
    else:
        for index,val in enumerate(all_drivers):
            if index>=total*8/10:
            #if index<10:
             im_path=str(train_path)+str(val)+'/'+str(images[index])
             fill_train(im_path,wigth,height,valid=True)
        y_valid,y_trainn=fill_train_target(all_drivers,valid=True)

    return y_valid,y_train

In [0]:
def show_image(image_path):
    im=cv2.imread(image_path,0)
    plt.imshow(im)

In [0]:
def fill_train(image_path,wigth,height,valid=False):
    if valid==False:
        img=cv2.imread(image_path,0)
        rotate = random.uniform(-10, 10)
        M = cv2.getRotationMatrix2D((img.shape[1]/2, img.shape[0]/2), rotate, 1)
        img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
        im = cv2.resize(img, (img_cols, img_rows), cv2.INTER_LINEAR)
        x_train.append(np.array(im, dtype=np.float32))
    else:
        img=cv2.imread(image_path,0)
        rotate = random.uniform(-10, 10)
        M = cv2.getRotationMatrix2D((img.shape[1]/2, img.shape[0]/2), rotate, 1)
        img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
        im = cv2.resize(img, (img_cols, img_rows), cv2.INTER_LINEAR)
        x_valid.append(np.array(im, dtype=np.float32))

In [0]:
def fill_train_target(all_drivers,valid=False):
    y_valid=[]
    y_train=[]
    if valid==False:
        lb = LabelEncoder()
        y_train = lb.fit_transform(all_drivers)
        y_train = np.array(y_train, dtype=np.float32)
        y_train = cv2.resize(y_train, (10,np.shape(x_train)[0]), cv2.INTER_LINEAR)
    else:
        lb = LabelEncoder()
        y_valid = lb.fit_transform(all_drivers)
        y_valid = np.array(y_valid, dtype=np.float32)
        y_valid = cv2.resize(y_valid, (10,np.shape(x_valid)[0]), cv2.INTER_LINEAR)

    return y_valid,y_train

In [0]:
y_valid,y_train=main_call()


In [0]:
#now we have x_train,y_train,x_valid,y_valid. that's good point!

In [0]:
x_train = np.array(x_train, dtype=np.uint8)
x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
x_train = x_train.astype('float32')
x_train /= 255

In [0]:
x_valid = np.array(x_valid, dtype=np.uint8)
x_valid = x_valid.reshape(x_valid.shape[0], 1, img_rows, img_cols)
x_valid = x_valid.astype('float32')
x_valid /= 255

In [0]:
y_train.reshape(1, 1,10,x_train.shape[0])
y_valid.reshape(1, 1,10,x_valid.shape[0])


In [0]:
#y_train = y_train.reshape((-1, 1))

In [0]:
#y_valid = y_valid.reshape((-1, 1))

In [0]:
def create_model():
    model = Sequential()
    model.add(Convolution2D(32, 5, 5, border_mode='same',name='conv1_1', input_shape=(color_type, img_rows, img_cols)))
    model.add(Activation("relu"))
    model.add(Convolution2D(32, 5, 5, border_mode='same',name='conv1_2'))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))
    model.add(Dropout(0.25))

    #2
    model.add(Convolution2D(64, 5, 5, border_mode='same',name='conv2_1'))
    model.add(Activation("relu"))
    model.add(Convolution2D(64, 5, 5, border_mode='same',name='conv2_2'))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))
    model.add(Dropout(0.25))

    #flatten
    model.add(Flatten())
    #model.add(Dense(512))
    #model.add(Activation("relu"))
    model.add(Dropout(0.5))

    model.add(Dense(10))
    model.add(Activation('softmax'))

    
    return model

In [0]:
model=create_model()   

In [0]:
from keras.optimizers import RMSprop
rms = RMSprop()
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.5, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=["accuracy"])


In [0]:
model.fit(x_train, y_train, batch_size=128, nb_epoch=epochs,
               verbose=1, validation_data=(x_valid,y_valid))
